<a href="https://colab.research.google.com/github/tmarathe-eng/Colab-notebook/blob/main/agentic_direct_orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# @title
# Fill in the blank answers

# This class allows for lazy evaluation of variables
class _Answer:
    def __init__(self, expr):
        self.expr = expr

    def _get_value(self):
        return eval(self.expr)

    def __repr__(self):
        return repr(self._get_value())

    def __str__(self):
        return str(self._get_value())

    def __eq__(self, other):
        return self._get_value() == other

    def __hash__(self):
        return hash(self._get_value())

    def __getitem__(self, key):
        return self._get_value()[key]

__YOUR_ANSWER_HERE_1 = _Answer('tool_call.function.name')
__YOUR_ANSWER_HERE_2 = _Answer('tool_name')
__YOUR_ANSWER_HERE_3 = _Answer('response.tool_calls')


# 🤖 Creating an Agentic Shopping Assistant

This notebook will go through all the steps to create an agentic shopping assistant through *direct orchestration*.

<br/>
<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/assistant.png" width="600">

Agents need access to an LLM. This notebook will use OpenAI.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/llm.png" width="120">

As part of [your homework]((https://colab.research.google.com/github/marta-manzin/agentic-shopping-assistant/blob/main/agentic_workshop_setup.ipynb)), you saved your OpenAI API key in a Colab Secret called "OPENAI_API_KEY".

Now, import the key into the notebook.

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

Imagine you want to build an agent.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agent.png" width="70">

With the following system prompt.

In [3]:
SYSTEM_PROMPT = """
You are a helpful assistant that adds and removes items from a shopping cart.

You have access to tools that let you:
1. Add grocery items
2. Remove grocery items
3. Inspect what is in the cart

Please do not ask me any follow-up questions after I make a request to you.
Just use the tools to satisfy my request.
"""

You start with an empty cart.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_empty.png" width="100">

In [4]:
MY_SHOPPING_CART = {}

You have a tool to add items.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_insert.png" width="100">

In [10]:
def add_to_cart(item_name: str, quantity: int=1) -> str:
  """Tool: "Add an grocery item to the shopping cart."."""
  try:
    if item_name not in MY_SHOPPING_CART:
      MY_SHOPPING_CART[item_name] = 0
    MY_SHOPPING_CART[item_name] += quantity
    return f"Added {quantity} {item_name}."
  except Exception as ex:
    return f"Failed to insert '{quantity}'. {ex!r}"


# TEST
add_to_cart("apples", 5) # Add an item to the cart
MY_SHOPPING_CART # Look into the cart

{'apples': 5}

A tool to remove items.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_remove.png" width="100">

In [11]:
def remove_from_cart(item_name: str) -> str:
  """Tool: Remove an item from the cart."""
  try:
    if item_name in MY_SHOPPING_CART:
      del MY_SHOPPING_CART[item_name]
      return f"Removed {item_name}."
    else:
      return f"{item_name} is not in the cart."
  except Exception as ex:
    return f"Failed to remove '{item_name}'. {ex!r}"


# TEST
remove_from_cart("kiwi") # Remove an item from the cart
MY_SHOPPING_CART # Look into the cart

{'apples': 5}

And a tool to see what's in the cart.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_view.png" width="120">

In [12]:
def whats_in_the_cart() -> str:
  """Tool: Get the contents of the cart."""
  try:
    empty = True
    for item, quantity in MY_SHOPPING_CART.items():
      if quantity > 0:
        empty = False
        break
    if empty:
      return "The cart is empty."
    else:
      return f"Here's the cart: {dict(sorted(MY_SHOPPING_CART.items()))}"
  except Exception as ex:
    return f"Failed to get cart contents. {ex!r}"


# TEST
# Look into the cart, this time using the tool
whats_in_the_cart()

"Here's the cart: {'apples': 5}"

A user sends the following prompt to your agent.

In [13]:
user_prompt = "Please add all the ingredients for fruit salad into my shopping cart."

What should the agent do?

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_arrows.png" width="800">

# 🔧 Tool Calling
To get started, the agent needs to tell the LLM which tools are available.

In [14]:
tools: list[dict] = [
    {
        "type": "function",
        "function": {
            "name": "add_to_cart",
            "description": "Add a grocery item to the shopping cart.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item_name": {
                        "type": "string",
                        "description": "The grocery item to be added."
                    },
                    "quantity": {
                        "type": "integer",
                        "description": "The quantity of the grocery item to be added."
                    },
                },
                "required": ["item_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "remove_from_cart",
            "description": "Remove an item from the cart.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item_name": {
                        "type": "string",
                        "description": "The grocery item to be removed."
                    },
                },
                "required": ["item_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "whats_in_the_cart",
            "description": "Get the contents of the shopping cart.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
]

If the LLM decides to run a tool, instead of responding with regular text, it will respond with a `tool_call` object.

In [15]:
from dataclasses import dataclass

# This is the tool_call format expected by OpenAI
@dataclass
class Function:
    name: str
    arguments: str

@dataclass
class ToolCall:
    id: str
    function: Function
    type: str

# This is the tool_call object
tool_call = ToolCall(
    id="call_abc123",
    function=Function(
        name="add_to_cart",
        arguments='{"item_name":"apples", "quantity": 5}'
    ),
    type="function"
)

Extract the tool name and the arguments from `tool_call`.

In [16]:
import json

tool_name = __YOUR_ANSWER_HERE_1
print(f"{tool_name=}")

arguments = json.loads(tool_call.function.arguments)
print(f"{arguments=}")

tool_name='add_to_cart'
arguments={'item_name': 'apples', 'quantity': 5}


Important! Verify that the tool requested is one of the available tools.



In [17]:
# List all available tools
allowed_tool_names = [tool["function"]["name"] for tool in tools]

# Check if the tool is available
if __YOUR_ANSWER_HERE_2 not in allowed_tool_names:
    print(f"Error: '{tool_name}' is not an allowed tool.")
else:
    print(f"'{tool_name}' is an allowed tool.")

'add_to_cart' is an allowed tool.


Call the tool and print its response.

In [18]:
tool_func = globals()[tool_name]
response = tool_func(**arguments)
response

'Added 5 apples.'

This is all you need to do to call a tool.

Let's combine all tool-calling steps in a single method called `execute`, for future use.

In [19]:
import json

def execute(tool_call) -> str:
    """Execute a tool call and return the result, if any."""
    # Extract the function name from the tool call
    function_name = tool_call.function.name

    # Parse the arguments from JSON string to dictionary
    arguments = json.loads(tool_call.function.arguments)

    # Important! Verify that the function is one of the allowed tools
    allowed_tool_names = [tool["function"]["name"] for tool in tools]
    if function_name not in allowed_tool_names:
        return f"Error: '{function_name}' is not an allowed tool."

    # Call the function with the unpacked arguments
    tool_func = globals()[function_name]
    response = tool_func(**arguments)

    # Return the tool's response
    return response

# 🔁 The Agentic Loop
Once the tool call is complete, the agent reports its outcome to the LLM.
The LLM might respond with more instructions! \
Then, the agent will operate in a `while True` loop, calling tools when the LLM requests them.  

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_loop.png" width="800">

All interactions between the agent, the LLM and the tools should be recorded in a message log. \
At the very beginning, the log only contains the system prompt and the user prompt.

In [20]:
messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": "Please add all the ingredients for fruit salad into my shopping cart."
    }
]

The following utility function prints out the log.

In [21]:
import textwrap

def print_log(messages, width=70):
    """Pretty print messages log."""
    # For each message in the log
    for i, msg in enumerate(messages):
        print(f"\n[{i}] Role: {msg['role']}")

        # Display message text if present
        if msg.get('content'):
            for line in str(msg['content']).split('\n'):
                wrapped = textwrap.fill(line, width=width, initial_indent='    ', subsequent_indent='    ')
                print(wrapped)

        # Display tool calls if present
        if msg.get('tool_calls'):
            for tc in msg['tool_calls']:
                func_name = tc.function.name
                func_args = tc.function.arguments
                print(f"    🔧 {func_name}({func_args})")


print_log(messages)


[0] Role: system

    You are a helpful assistant that adds and removes items from a
    shopping cart.

    You have access to tools that let you:
    1. Add grocery items
    2. Remove grocery items
    3. Inspect what is in the cart

    Please do not ask me any follow-up questions after I make a
    request to you.
    Just use the tools to satisfy my request.


[1] Role: user
    Please add all the ingredients for fruit salad into my shopping
    cart.


The agent asks the LLM what to do next.

In [22]:
import openai
client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto"
).choices[0].message

# Display the raw tool call in the agent's response
tool_calls_data = [tc.model_dump() for tc in response.tool_calls]
print(json.dumps(tool_calls_data, indent=2))

[
  {
    "id": "call_hVtn12PR0MFiJgu5jnPFZBnd",
    "function": {
      "arguments": "{\"item_name\": \"apples\", \"quantity\": 2}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_or5bhdUJulUfLTUL4h0dGa5l",
    "function": {
      "arguments": "{\"item_name\": \"bananas\", \"quantity\": 2}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_jyuREIrf3JcyLOhWwLMEKWVZ",
    "function": {
      "arguments": "{\"item_name\": \"grapes\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_mjjDs7OTDquDhsvLtfUXn7nn",
    "function": {
      "arguments": "{\"item_name\": \"oranges\", \"quantity\": 2}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_bV60wPLHDQAckJlXtdprK8Z4",
    "function": {
      "arguments": "{\"item_name\": \"strawberries\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type": "function"
  }
]


The agent updates the message log with the LLM's response.

In [23]:
messages.append({
    "role": "assistant",
    "content": response.content,
    "tool_calls": response.tool_calls
})

print_log(messages)


[0] Role: system

    You are a helpful assistant that adds and removes items from a
    shopping cart.

    You have access to tools that let you:
    1. Add grocery items
    2. Remove grocery items
    3. Inspect what is in the cart

    Please do not ask me any follow-up questions after I make a
    request to you.
    Just use the tools to satisfy my request.


[1] Role: user
    Please add all the ingredients for fruit salad into my shopping
    cart.

[2] Role: assistant
    🔧 add_to_cart({"item_name": "apples", "quantity": 2})
    🔧 add_to_cart({"item_name": "bananas", "quantity": 2})
    🔧 add_to_cart({"item_name": "grapes", "quantity": 1})
    🔧 add_to_cart({"item_name": "oranges", "quantity": 2})
    🔧 add_to_cart({"item_name": "strawberries", "quantity": 1})


The agent executes the first tool call.

In [24]:
outcome = execute(__YOUR_ANSWER_HERE_3[0])
outcome

'Added 2 apples.'

The agent appends the outcome of the tool call to the message log.

In [25]:
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(outcome)
})
print_log(messages)


[0] Role: system

    You are a helpful assistant that adds and removes items from a
    shopping cart.

    You have access to tools that let you:
    1. Add grocery items
    2. Remove grocery items
    3. Inspect what is in the cart

    Please do not ask me any follow-up questions after I make a
    request to you.
    Just use the tools to satisfy my request.


[1] Role: user
    Please add all the ingredients for fruit salad into my shopping
    cart.

[2] Role: assistant
    🔧 add_to_cart({"item_name": "apples", "quantity": 2})
    🔧 add_to_cart({"item_name": "bananas", "quantity": 2})
    🔧 add_to_cart({"item_name": "grapes", "quantity": 1})
    🔧 add_to_cart({"item_name": "oranges", "quantity": 2})
    🔧 add_to_cart({"item_name": "strawberries", "quantity": 1})

[3] Role: tool
    Added 2 apples.



Let's combine all steps of the agentic logic into a single loop.

In [26]:
def submit_request(
    user_prompt: str,
    verbose: bool = True
    ):
    """Submit a request to the agent and run any tools it calls."""
    # Initialize the log
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    while True:

        # Ask the agent what to do next
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        ).choices[0].message

        # Update the log with the agent's response
        messages.append({
            "role": "assistant",
            "content": response.content,
            "tool_calls": response.tool_calls
        })

        # If agent did not call any tools, we are done
        if not response.tool_calls:
            if verbose:
              print(f"\n⭐ {whats_in_the_cart()}")
            break

        # Execute all tool calls
        for tool_call in response.tool_calls:
            if verbose:
              print(f"\n🔧 The agent is calling a tool: "
                  f"{tool_call.function.name}"
                  f"({json.loads(tool_call.function.arguments)})")

            # Append the outcome to the log
            outcome = execute(tool_call)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(outcome)
            })

## Let's test the agent!

Start with an empty cart.

In [27]:
MY_SHOPPING_CART = {}

Ask the agent to add some items.

In [28]:
submit_request("Please add all the ingredients for lasagna into the cart.")


🔧 The agent is calling a tool: add_to_cart({'item_name': 'lasagna noodles', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'ground beef', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'onion', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'garlic', 'quantity': 2})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'tomato sauce', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'tomato paste', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'ricotta cheese', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'mozzarella cheese', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Parmesan cheese', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'egg', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'dried basil', 'quantity': 1})

🔧 The agent is calli

Ask the agent to remove some items.

In [29]:
submit_request("Please remove all meat from the cart.")


🔧 The agent is calling a tool: whats_in_the_cart({})

🔧 The agent is calling a tool: remove_from_cart({'item_name': 'ground beef'})

⭐ Here's the cart: {'Parmesan cheese': 1, 'black pepper': 1, 'dried basil': 1, 'dried oregano': 1, 'egg': 1, 'garlic': 2, 'lasagna noodles': 1, 'mozzarella cheese': 1, 'onion': 1, 'ricotta cheese': 1, 'salt': 1, 'tomato paste': 1, 'tomato sauce': 1}


This "manual" way of managing an agent is called *direct orchestration*. \
It's helpful to understand how an agent works under the hood! \
In the [next notebook](https://colab.research.google.com/github/marta-manzin/agentic-shopping-assistant/blob/main/agentic_frameworks_and_mcp.ipynb), we will create a similar agent using an open-source framework and an MCP server.

###